In [16]:
from __future__ import division
from imret.dataset import Dataset
from imret.query import Annotation
import pandas as pd

In [17]:
import caffe

In [18]:
net = caffe.Net('/Users/danilonunes/workspace/caffe-libs/sceneparsing/models/deploy_DilatedNet.prototxt',
                '/Users/danilonunes/workspace/caffe-libs/sceneparsing/models/DilatedNet_iter_120000.caffemodel',
                caffe.TEST)

In [22]:
df = pd.read_csv('/Users/danilonunes/workspace/caffe-libs/sceneparsing/objectInfo150.txt', sep='\t')
df.head()

,Idx,Ratio,Train,Val,Name
0,1,0.1576,11664,1172,wall
1,2,0.1072,6046,612,"building, edifice"
2,3,0.0878,8265,796,sky
3,4,0.0621,9336,917,"floor, flooring"
4,5,0.0480,6678,641,tree


In [23]:
names = [name.split(',')[0] for name in set(df.Name)]
print(len(names))

150


In [87]:
import cv2
import cytoolz
import re

In [63]:
is_train = False
if is_train:
    dataset = Dataset('../data/datasets/', 'train', '../data/images/')
    annot = Annotation('../data/query/train_anno/')
    
else:
    dataset = Dataset('../data/datasets2/', 'test', '../data/images/')
    annot = Annotation('../data/query/test_anno/')
    
images = set(annot.imgs.keys()) & set(dataset.images)

In [64]:
areas = {}
total = []
norm = {}

for imname in images:
    imarray = dataset.get_im_array(imname)
    if imarray is None:
        continue
        
    else:
        w, h = imarray.shape[:2]
    
    ground_truth = {k: v for k, v in dataset.ground_truth(imname).items() if re.match('\D+',k).group() in annot.normalized_names}
    for obj, contour in ground_truth.items():
        name = cp.class_names.get(obj, obj)
        if name not in areas:
            areas[name] = 0
            norm[name] = 0
        areas[name] += cv2.contourArea(contour) 
        norm[name] += (cv2.contourArea(contour) / (w*h))
        total.append(name)

In [65]:
freq = cytoolz.frequencies(total)

In [66]:
if is_train:
    train = {}
    for k, v in sorted(freq.items(), key=lambda x: x[0]):
        train.setdefault('name', []).append(k)
        train.setdefault('count', []).append(v)
        train.setdefault('area', []).append(areas[k])
        train.setdefault('norm', []).append(norm[k])
    df = pd.DataFrame(train)
else:
    test = {}
    for k, v in sorted(freq.items(), key=lambda x: x[0]):
        test.setdefault('name', []).append(k)
        test.setdefault('count', []).append(v)
        test.setdefault('area', []).append(areas[k])
        test.setdefault('norm', []).append(norm[k])
    df2 = pd.DataFrame(test)

In [67]:
merged = pd.merge(df, df2, on='name')
merged.head()

,area_x,count_x,name,norm_x,area_y,count_y,norm_y
0,1708749.5,31,airplane,1.465263,1712876.5,28,0.915717
1,2280386.0,79,armchair,3.551011,978449.5,77,3.063309
2,2264473.0,57,balcony,3.680047,1238547.0,56,3.718344
3,530225.5,19,bar,2.564089,186530.0,13,1.211677
4,304565.5,40,basket,0.437681,249371.5,51,0.718224


In [69]:
merged['relation_count'] = merged[['count_x', 'count_y']].apply(lambda x: x['count_x'] / float(x['count_y']), axis=1)
merged['relation_area'] = merged[['area_x', 'area_y']].apply(lambda x: x['area_x'] / float(x['area_y']), axis=1)
merged['relation_norm'] = merged[['norm_x', 'norm_y']].apply(lambda x: x['norm_x'] / float(x['norm_y']), axis=1)


In [70]:
merged[['name', 'count_x', 'area_x', 'norm_x', 'count_y', 'area_y', 'norm_y', 'relation_count', 'relation_area', 'relation_norm']].head()

,name,count_x,area_x,norm_x,count_y,area_y,norm_y,relation_count,relation_area,relation_norm
0,airplane,31,1708749.5,1.465263,28,1712876.5,0.915717,1.107143,0.997591,1.600126
1,armchair,79,2280386.0,3.551011,77,978449.5,3.063309,1.025974,2.330612,1.159208
2,balcony,57,2264473.0,3.680047,56,1238547.0,3.718344,1.017857,1.828330,0.989700
3,bar,19,530225.5,2.564089,13,186530.0,1.211677,1.461538,2.842575,2.116149
4,basket,40,304565.5,0.437681,51,249371.5,0.718224,0.784314,1.221332,0.609394


In [72]:
for _, row in merged.iterrows():
    print "\\textit{{{name}}} & {freq_train} & {area_train:.2f} & {freq_test} & {area_test:.2f} & {relation_count:.2f} & {relation_area:.2f} & \\\\".format(name=row['name'].title(),
                                                                                                                            freq_train=row['count_x'],
                                                                                                                            area_train=row['norm_x'] / row['count_x'],
                                                                                                                            freq_test=row['count_y'],
                                                                                                                            area_test=row['norm_y'] / row['count_y'],
                                                                                                                            relation_count=row['relation_count'],
                                                                                                                            relation_area=row['relation_norm'] / row['relation_count'])

\textit{Airplane} & 31 & 0.05 & 28 & 0.03 & 1.11 & 1.45 & \\
\textit{Armchair} & 79 & 0.04 & 77 & 0.04 & 1.03 & 1.13 & \\
\textit{Balcony} & 57 & 0.06 & 56 & 0.07 & 1.02 & 0.97 & \\
\textit{Bar} & 19 & 0.13 & 13 & 0.09 & 1.46 & 1.45 & \\
\textit{Basket} & 40 & 0.01 & 51 & 0.01 & 0.78 & 0.78 & \\
\textit{Bench} & 56 & 0.03 & 79 & 0.03 & 0.71 & 1.03 & \\
\textit{Book} & 37 & 0.00 & 48 & 0.01 & 0.77 & 0.65 & \\
\textit{Bookcase} & 29 & 0.19 & 26 & 0.22 & 1.12 & 0.87 & \\
\textit{Bottle} & 46 & 0.00 & 44 & 0.00 & 1.05 & 1.27 & \\
\textit{Box} & 60 & 0.01 & 80 & 0.02 & 0.75 & 0.71 & \\
\textit{Building} & 930 & 0.21 & 941 & 0.23 & 0.99 & 0.89 & \\
\textit{Cabinet} & 68 & 0.06 & 77 & 0.07 & 0.88 & 0.84 & \\
\textit{Car} & 359 & 0.01 & 289 & 0.01 & 1.24 & 0.82 & \\
\textit{Chair} & 321 & 0.02 & 348 & 0.03 & 0.92 & 0.87 & \\
\textit{Closet} & 18 & 0.15 & 22 & 0.18 & 0.82 & 0.83 & \\
\textit{Curtain} & 139 & 0.08 & 165 & 0.09 & 0.84 & 0.86 & \\
\textit{Desk} & 67 & 0.08 & 89 & 0.08 & 0.75 & 0.9

In [78]:
print merged.mean()['count_y']

258.128205128


In [86]:
print merged.mean()['relation_norm'] / merged.mean()['relation_count']

1.00050273065


In [134]:
29.29 / 228.5

0.12818380743982494

In [80]:
merged.mean()['relation_count']

0.98267267471132669

In [83]:
merged[['name', 'count_x' , 'norm_x']].tail()

,name,count_x,norm_x
34,van,68,0.862464
35,vase,82,0.381452
36,wall,877,184.092173
37,water,85,17.602346
38,window,775,25.732346
